## Authenticate to Google Earth Engine

In [1]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

In [8]:
def get_feature_collection(asset_id):
    """Check if an asset exists, and load it as a FeatureCollection if it does.
    Otherwise, return an empty FeatureCollection.
    
    Args:
        asset_id (str): The Earth Engine asset ID.
        
    Returns:
        ee.FeatureCollection: The loaded FeatureCollection or an empty one.
    """
    try:
        # Get asset information to check existence
        ee.data.getAsset(asset_id)
        print(f"Asset '{asset_id}' exists. Loading FeatureCollection.")
        return ee.FeatureCollection(asset_id)
    except Exception as e:
        print(f"Asset '{asset_id}' does not exist. Returning empty FeatureCollection.")
        return ee.FeatureCollection([])

In [ ]:
filename_prefix = "Area_forested"
suffix = filename_prefix.split("_")[-1]
mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

def get_class_and_mapping(vectors, vector_name):
    vector_of_class = vectors.filter(ee.Filter.eq('class', vector_name))
    vector_of_class_with_map = vector_of_class.map(lambda f: f.set('class', mapping[vector_name]))
    return vector_of_class, vector_of_class_with_map

all = get_feature_collection("projects/ee-raman/assets/all_" + suffix)
farm, farm_with_map = get_class_and_mapping(all, "farm")
scrubland, scrubland_with_map = get_class_and_mapping(all, "scrubland")
plantation, plantation_with_map = get_class_and_mapping(all, "plantation")
rest, rest_with_map = get_class_and_mapping(all, "rest")

ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + suffix)

training_features = farm_with_map.merge(scrubland_with_map).merge(plantation_with_map)

training_data = ts_data.sampleRegions(
    collection=training_features,
    properties=['class'],
    scale=10  # Adjust scale based on resolution
)
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training_data,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)

classified = ts_data.classify(classifier)
def assign_mode_label(feature):
    class_values = classified.reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=feature.geometry(),
        scale=30,  # Adjust scale as per resolution
        bestEffort=True
    )
    return feature.set('class', class_values.get('classification'))

# Apply function to test features
rest_with_labels = rest_with_map.map(assign_mode_label).filter(ee.Filter.notNull(['class']))
rest_with_labels = rest_with_labels.map(lambda x: x.set('class', reversed_ee_mapping.get(ee.Number(x.get("class")).int())))
all_with_labels = rest_with_labels.merge(farm).merge(scrubland)

Asset 'projects/ee-raman/assets/all_forested' exists. Loading FeatureCollection.


In [50]:
task = ee.batch.Export.table.toAsset(
    collection=all_with_labels,
    description='Classification',
    assetId="projects/ee-raman/assets/refined_all_" + suffix
)

# Start the task
task.start()